In [18]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import pandas as pd
import os

In [19]:
def test_sheet(num):
    sheet_num = num
    ws = wb.worksheets[sheet_num]
    test_str = False 
    
    start_row = False
    start_col = False
    
    company = {}
    
    for cell in ws['B']:
        if 'Company Name' in str(cell.value):
            address_row = cell.row + 1
            address_cell = 'B{}'.format(address_row)
            address_str = ws[address_cell].value
            
            address_lines = address_str.splitlines()
            state_test = address_lines.index(', CA')
            city = state_test - 1
            
            company['city'] = address_lines[city]

            
            company_name = address_lines[0]
            
            address = address_lines[1:]
            
            company['name'] = company_name
            company['address'] = address
            
            start_row = cell.row #the with Company Name
            start_col = get_column_letter(cell.column)
            
    for i in ws[start_row]:
        if 'Employees' in str(i.value): #gets the number of employees affected
            new_row = i.row
            new_col = get_column_letter(i.column)
            new_row = new_row + 1
            next_cell = '{}{}'.format(new_col, new_row)
            new_str = ws[next_cell].value
            company['num_employees_affected'] = new_str
    
    for i in ws[start_row]:
        if 'Layoff' in str(i.value): #gets the layoff date
            new_row = i.row
            new_col = get_column_letter(i.column)
            new_row = new_row + 1
            next_cell = '{}{}'.format(new_col, new_row)
            new_str = ws[next_cell].value
            company['layoff_date'] = new_str
    
    union_row = False
            
    for cell in ws[start_col]:
        if 'Union Representation' in str(cell.value):
            union_row = cell.row
            union_col_num = cell.column
            union_col = get_column_letter(union_col_num)
            
            
    for i in ws[union_row]:
        if type(i.value) == str:
            union_col = get_column_letter(i.column)
            union_row = i.row
            union_cell = '{}{}'.format(union_col, union_row)
            union_info = ws[union_cell].value
            company['union'] = union_info
    

    for cell in ws[start_col]:
        if 'representing' in str(cell.value):
            next_row = cell.row + 1
            next_cell = '{}{}'.format(start_col, next_row)
            next_str = ws[next_cell].value
            rep_str = ''
            while 'Job' not in next_str:
                rep_str = rep_str + ws[next_cell].value
                next_row += 1
                next_cell = '{}{}'.format(start_col, next_row)
                next_str = ws[next_cell].value
            company['union_rep_info'] = rep_str
    
    
    new_row = False
    new_col = False
    
    workforce_str = False
    
    end_col = False
    end_row = False
    
    for i in ws[start_row]:
        if 'Local Workforce' in str(i.value): #gets the local workforce investment area
            new_row = i.row
            new_col = get_column_letter(i.column)
            end_col = new_col
            new_row = new_row + 1
            next_cell = '{}{}'.format(new_col, new_row)
            new_str = ws[next_cell].value
            company['local_workforce_investment_area'] = new_str
    
    notice = False
    
    start2 = False
    start2_row = False
    start2_col = False
    for i in ws['B']:
        if 'Contact' in str(i.value):
            start2_row = i.row
            start2_col = get_column_letter(i.column)

    for i in ws[start2_row]:
        if 'Date' in str(i.value):
            notice = str(i.value)
            new_row = i.row
            new_col = i.column
            new_col = new_col + 1
            notice_cell = '{}{}'.format(new_col, new_row)
            notice = str(notice_cell)
    

    for i in ws[start_row]:
        if 'Local Workforce' in str(i.value): #gets the local workforce investment area
            new_row = i.row
            new_col = get_column_letter(i.column)
            end_col = new_col
            new_row = new_row + 1
            next_cell = '{}{}'.format(new_col, new_row)
            new_str = ws[next_cell].value
            company['local_workforce_investment_area'] = new_str
    
    
    for i in ws[end_col]:
        date_row = start_row + 2
        date_cell = '{}{}'.format(end_col, date_row)
        date_str = ws[date_cell].value
        company['date_notice_received'] = date_str
        type_row = date_row + 1
        type_cell = '{}{}'.format(end_col, type_row)
        type_str = ws[type_cell].value
        company['layoff_or_closure'] = type_str
        severance_row = type_row + 1
        severance_cell = '{}{}'.format(end_col, severance_row)
        severance_str = ws[severance_cell].value
        company['severance'] = severance_str  
        bump_row = severance_row + 1
        bump_cell = '{}{}'.format(end_col, bump_row)
        bump_str = ws[bump_cell].value
        company['bumping_rights'] = bump_str  
    
    jobs = {}
    
    start3 = False
    start3_row = False
    start3_col = False

    for i in ws['B']:
        if 'Job Title' in str(i.value): #finds and stores the index of the Job Titles header
            start3 = str(i.value)
            start3_row = i.row
            start3_col = get_column_letter(i.column)

    positions_header = False
    positions_col = False
    positions_row = False

    for pos in ws[start3_row]:
        if '#' in str(pos.value):
            positions_col = get_column_letter(pos.column)
            positions_row = pos.row #this sets positions_row as the the row with the header

    #now get the first job title
    job_row = start3_row + 1
    for i in ws['B']:
        job_cell = '{}{}'.format(start3_col, job_row)
        job_str = ws[job_cell].value
        positions_row = job_row
        positions_cell = '{}{}'.format(positions_col, positions_row)
        positions_str = ws[positions_cell].value
        jobs[job_str] = positions_str
        job_row += 1

    company['jobs'] = jobs
            
      
    return company

In [25]:
directory = '/Users/vochavillo/Desktop/Working_Projects/warn/california/ca-warn-reports-2020-03-16/'

df = pd.DataFrame()
company_list = []

for filename in os.listdir(directory):
    if filename.endswith('xlsx'):
        dir_full = directory + filename
        
        wb = load_workbook(dir_full)
        
        sheet_counter = 0

        sheet_index = []

        for sheet in wb.worksheets:
            sheet_index.append(sheet)
            
        last_sheet = len(sheet_index) - 1

        while sheet_counter <= last_sheet:
            hold_dict = test_sheet(sheet_counter)
            company_list.append(hold_dict)
            sheet_counter += 1
        
        df = pd.DataFrame(company_list)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5450 entries, 0 to 5449
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   city                             5450 non-null   object        
 1   name                             5450 non-null   object        
 2   address                          5450 non-null   object        
 3   num_employees_affected           5450 non-null   int64         
 4   layoff_date                      5450 non-null   datetime64[ns]
 5   union                            5450 non-null   object        
 6   union_rep_info                   5450 non-null   object        
 7   local_workforce_investment_area  5450 non-null   object        
 8   date_notice_received             5450 non-null   datetime64[ns]
 9   layoff_or_closure                5450 non-null   object        
 10  severance                        5450 non-null   object     

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5450 entries, 0 to 5449
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   city                             5450 non-null   object        
 1   name                             5450 non-null   object        
 2   address                          5450 non-null   object        
 3   num_employees_affected           5450 non-null   int64         
 4   layoff_date                      5450 non-null   datetime64[ns]
 5   union                            5450 non-null   object        
 6   union_rep_info                   5450 non-null   object        
 7   local_workforce_investment_area  5450 non-null   object        
 8   date_notice_received             5450 non-null   datetime64[ns]
 9   layoff_or_closure                5450 non-null   object        
 10  severance                        5450 non-null   object     

In [29]:
df.to_csv('california_warn_cleaned_updated_2020-04-30.csv', index=False)

In [30]:
df.to_json('california_warn_cleaned_updated_2020-04-30.json')

In [32]:
warn_json = pd.read_json('california_warn_cleaned_updated_2020-04-30.json')

In [37]:
warn_json

,city,name,address,num_employees_affected,layoff_date,union,union_rep_info,local_workforce_investment_area,date_notice_received,layoff_or_closure,severance,bumping_rights,jobs
0,Chatsworth,Albertelli Law,"[9200 Oakdale Avenue, Chatsworth, , CA, 91311]",66,1418601600000,No,,City of Los Angeles ...,1413936000000,Layoff,Employer did not State,No,"{'Bankruptcy Paralegal': 1, 'Foreclosure Speci..."
1,Los Angeles,"Amcor Rigid Plastics USA, Inc.","[2010 Yates Avenue, Los Angeles, , CA, 90040]",112,1411689600000,No,,Los Angeles County WDACS ...,1407974400000,Layoff,Employer did not State,No,"{'Lead': 6, 'Machine Operator': 51, 'Maintenan..."
2,Edwards,Arcata Associates Inc.,"[NASA Armstrong Flight Research Center, 4846 L...",101,1414713600000,No,,Los Angeles County WDACS ...,1412035200000,Layoff,Employer did not State,No,"{'Administrator, COMSEC Count': 1, 'Administra..."
3,Torrance,American Medical Response of Southern California,"[20101 Hamilton Avenue, Suite 300, Torrance, ...",120,1415923200000,No,,"City of Long Beach, Pac Gateway Innovation Net...",1406592000000,Closure,Employer did not State,No,"{'Account Receivable Followup Manager': 1, 'Ac..."
4,Wheatland,ARAMARK,"[2677 FORTY MILE ROAD, Wheatland, , CA, 95692]",152,1419984000000,No,,North Central Counties Consortium ...,1419206400000,Closure,Employer did not State,No,"{'BARTENDER': 25, 'BEVERAGE ATTENDANT': 25, 'C..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5445,San Francisco,"GREE International, Inc.","[185 Berry Street, Suite 590, San Francisco, ...",114,1437350400000,Employer did not State,,Office of Economic and Workforce Development S...,1442361600000,Layoff,Employer did not State,Employer did not State,"{'EDNP': 114, 'None': None}"
5446,Los Angeles,FAMIMA Corporation (700 Wilshire store),"[700 Wilshire Blvd, Los Angeles, , CA, 90017]",10,1446163200000,Employer did not State,,City of Los Angeles ...,1441324800000,Layoff,Employer did not State,Employer did not State,"{'EDNP': 10, 'None': None}"
5447,Torrance,FAMIMA Corporation,"[20000 Mariner Avenue, Suite 100, Torrance, ,...",9,1446163200000,Employer did not State,,"City of Long Beach, Pac Gateway Innovation Net...",1441324800000,Layoff,Employer did not State,Employer did not State,"{'EDNP': 9, 'None': None}"
5448,Citrus Heights,Kmart,"[8501 Auburn Blvd, Citrus Heights, , CA, 95610]",86,1459641600000,No,,Sacramento Employment and Training Agency ...,1453766400000,Closure,Employer did not State,No,"{'EDNP': 86, 'None': None}"
